<a href="https://colab.research.google.com/github/MGino-BR/EntendAI/blob/main/EntendAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Projeto EntendAI

Prepare-se para desvendar os mistérios da comunicação, sem fronteiras! Já se pegou boiando em uma conversa com um amigo de outro estado? Ou tentando decifrar o que seu sobrinho Gen Z quis dizer com "cringe"? Se a resposta é sim, então EntendAI é o seu novo melhor amigo!

Este incrível chatbot foi criado para te ajudar a navegar pelas fascinantes águas da linguagem regional brasileira e pelos códigos comunicacionais das diferentes gerações. Com o EntendAI, você poderá traduzir gírias de cada canto do Brasil e finalmente entender se o "barril" do seu colega carioca é a mesma coisa que o "barril" do seu vizinho baiano.

E tem mais! Para facilitar a comunicação com turistas, estrangeiros e aquela galera que está aprendendo português, o EntendAI também traduz o significado desses termos regionais para diversas outras línguas! Agora, entender o "arretado" do nordestino ou o "tri legal" do sul nunca foi tão fácil, não importa de onde você seja! E não para por aí! Desvende os segredos da comunicação entre gerações, traduzindo aquele "se joga" da sua tia Baby Boomer para o "sextou" do seu primo Alpha, e quem sabe, até para o "let's go" de um amigo gringo!

Chega de mal-entendidos, seja você brasileiro ou de qualquer canto do mundo! Mergulhe no universo EntendAI e torne suas conversas mais ricas, acessíveis e divertidas!

## Configurações Iniciais



Instalação de biblioteca Python dos modelos IA do Google.

---



In [139]:
# Instala a biblioteca Python dos modelos de IA generativa do Google
!pip -q install google-genai

Configuração da API Key do Google Gemini.

---

In [140]:
# Configura a API Key do Google Gemini
import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

Configuração da SDK do Gemini.

---

In [141]:
# Configura o cliente da SDK do Gemini
from google import genai

client = genai.Client()

MODEL_ID = "gemini-2.0-flash"

Configuração de exibição de texto formatado no Colab.

---

In [142]:
# Função auxiliar para exibir texto formatado em Markdown no Colab
from IPython.display import HTML, Markdown, display
import textwrap

def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

## Configuração inicial para utilização dos agentes de IA

Instalação de biblioteca ADK

---

In [143]:
# Instalar Framework ADK de agentes do Google
!pip install -q google-adk

Importação das ferramentas para os agentes de IA

---

In [144]:
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types  # Para criar conteúdos (Content e Part)
from datetime import date
import textwrap # Para formatar melhor a saída de texto
from IPython.display import display, Markdown # Para exibir texto formatado no Colab
import requests # Para fazer requisições HTTP
import warnings

warnings.filterwarnings("ignore")

Declaração da função de chamada dos agentes de IA

---

In [145]:
# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final
def call_agent(agent: Agent, message_text: str) -> str:
    # Cria um serviço de sessão em memória
    session_service = InMemorySessionService()
    # Cria uma nova sessão (você pode personalizar os IDs conforme necessário)
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    # Cria um Runner para o agente
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    # Cria o conteúdo da mensagem de entrada
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    # Itera assincronamente pelos eventos retornados durante a execução do agente
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response

## Configuração dos Agentes de IA

Agente 1: Tradutor Regional do Brasil

---

In [146]:
# Agente 1: Especialista em Tradução Regional do Brasil
def agente_tradutor_regional(opcao, termo, estado, lingua_destino):

    tradutor_regional = Agent(
        name = "agente_tradutor_regional",
        model = "gemini-2.0-flash",
        instruction = """
        Você é um assistente especialista de tradução regional do Brasil para qualquer lingua. A sua tarefa é usar a ferramenta de busca do Google (google_search)
        para procurar pelo significado atualizado da palavra ou expressões regionais solicitadas de acordo com a região (estado brasileiro) e retornar
        o resultado traduzido para a lingua selecionada.
        Certifique-se de revisar o resultado da busca e retornar apenas a tradução precisa e atualizada da palavra ou expressão regional solicitada.
        Suas respostas devem ser claras, sucintas e com no máximo 5 significados.
        Se não encontrar termos relevantes, informe que não conseguiu encontrar a tradução.
        Se o estado informado não for válido, informe que não conseguiu encontrar a região.
        Se a lingua informada não for válida, informe que não conseguiu encontrar a lingua.
        O seu tom deve ser lúdico e divertido.
        Nunca demore mais que 05 segundos para responder.
        Evite qualquer texto introdutório ou repetição do pedido original.
        Se a lingua de destino for português, retorne o significado ou uma breve explicação do termo ou expressão regional na própria língua portuguesa,
        se relevante. Para a listagem de termos regionais, apenas liste os termos relevantes da região.
        """,
        description = "Agente que busca e retorna a tradução de palavras e expressões regionais baseado na regiãoe na lingua final selecionada.",
        tools = [google_search],
    )

    if opcao == '1':
      entrada_do_agente_tradutor_regional = f"Traduza a seguinte palavra ou expressão regional brasileira do {estado} para {lingua_destino}: '{termo}'"
    elif opcao == '2':
      entrada_do_agente_tradutor_regional = f"Liste os 20 termos ou expressões regionais mais relevantes da região '{estado}'do Brasil e traduza-os para '{lingua_destino}'."

    traducao_regional = call_agent(tradutor_regional, entrada_do_agente_tradutor_regional)
    return traducao_regional

Agente 2: Especialista em Tradução Integeracional

---

In [147]:
# Agente 2: Especialista em Tradução entre Gerações
def agente_tradutor_geracao(termo, geracao_origem, geracao_destino):

    agente_tradutor_geracao = Agent(
        name = "agente_tradutor_geracao",
        model = "gemini-2.0-flash",
        instruction = """
        Você é um assistente especialista de tradução entre gerações. A sua tarefa é usar a ferramenta de busca do Google (google_search)
        para procurar pelo significado atualizado da palavra ou expressões da geração solicitada e retornar
        o resultado traduzido usando termos que a geração destino entenda.
        Certifique-se de revisar o resultado da busca e retornar apenas a tradução precisa e atualizada da palavra ou expressão solicitada.
        Suas respostas devem ser claras, sucintas.
        Se não encontrar termos relevantes, informe que não conseguiu encontrar a tradução.
        O seu tom deve ser lúdico e divertido.
        Nunca demore mais que 05 segundos para responder.
        Evite qualquer texto introdutório ou repetição do pedido original.
        Se a expressão informada não pertencer a geração ORIGEM também informada, retorne que a expressão não é conhecida por essa geração. Depois diga qual é a geração
        correta e der uma pequena explicação na geração DESTINO informada.
        """,
        description = "Agente que busca e retorna a tradução de palavras e expressões de uma geração para explicar na forma que a geração destino selecionada entende.",
        tools = [google_search],
    )

    entrada_do_agente_tradutor_geracao = f"Explique a seguinte palavra ou expressão da Geração {geracao_origem} na linguagem da Geração {geracao_destino}: '{termo}'"

    traducao_geracao = call_agent(agente_tradutor_geracao, entrada_do_agente_tradutor_geracao)
    return traducao_geracao

## Código do Projeto
---

In [ ]:
def main():
  #Função principal para executar o chatbot.
  print("Bem-vindo ao EntendAI!")

  #Seleção das Opções
  while True:
    print("\nSelecione o que deseja fazer:")
    print("1. Traduzir termo regional")
    print("2. Ver lista de termos regionais")
    print("3. Traduzir termos entre Gerações")
    print("4. Sair")

    opcao = input("Digite o número da sua escolha: ")

    #Traduzindo termo regional
    if opcao == '1':
      estado = input("\nDigite a região (estado brasileiro): ")
      lingua_destino = input("\nDigite a língua para tradução: ")
      continuar_traduzindo_regional = 's'  # Forçando o valor inicial para "s"
      while continuar_traduzindo_regional == 's':
        termo = input("\nDigite o termo regional: ")
        resultado_termo = agente_tradutor_regional(opcao, termo, estado, lingua_destino)
        print("\n🔍 Resultado")
        display(to_markdown(resultado_termo))
        continuar_traduzindo_regional = input("\nDeseja traduzir outra palavra ou expressão regional? (s/n): ")
        while continuar_traduzindo_regional.lower() not in ['s', 'n']: #verificação de entradas válidas
              print("\nOpção inválida. Por favor, tente novamente.\n")
              continuar_traduzindo_regional = input("\nDeseja traduzir outra palavra ou expressão regional? (s/n): ")
        if continuar_traduzindo_regional.lower() != 's':
              print("\nO que deseja fazer agora?\n")
              break # Sai do loop de tradução regional e volta para a seleção principal

    #Visualizando lista de termos regionais
    elif opcao == '2':
        estado = input("\nDigite a região (estado brasileiro): ")
        lingua_destino = input("\nDigite a língua para tradução: ")
        termo='' # Forçando o valor inicial para ''
        resultado_lista = agente_tradutor_regional(opcao, termo, estado, lingua_destino)
        print(f"\n📝 Lista Regional")
        display(to_markdown(resultado_lista))
        print("\nO que deseja fazer agora?\n")

    #Traduzindo entre gerações
    elif opcao == '3':
      geracao_origem = input("\nDigite a geração de ORIGEM (Alpha, Z, X, Y, Baby Boomers): ")
      geracao_destino = input("\nDigite a geração de DESTINO (Alpha, X, Y, Baby Boomers): ")
      continuar_traduzindo_geracao = 's'  # Forçando o valor inicial para "s"
      while continuar_traduzindo_geracao == 's':
        termo = input("\nDigite a palavra ou expressão: ")
        resultado_geracao = agente_tradutor_geracao(termo, geracao_origem, geracao_destino)
        print(f"\n✍️ Resultado da Tradução Intergeracional")
        display(to_markdown(resultado_geracao))
        continuar_traduzindo_geracao = input("\nDeseja traduzir outra expressão entre essas gerações? (s/n): ")
        while continuar_traduzindo_geracao.lower() not in ['s', 'n']: #verificação de entradas válidas
              print("\nOpção inválida. Por favor, tente novamente.\n")
              continuar_traduzindo_geracao = input("\nDeseja traduzir outra expressão entre essas gerações? (s/n): ")
        if continuar_traduzindo_geracao.lower() != 's':
              print("\nO que deseja fazer agora?\n")
              break # Sai do loop de tradução regional e volta para a seleção principal

    #saindo e encerrando o programa.
    elif opcao == '4':
      print("\n Obrigado por usar o EntendAI!")
      break

    #verificação de entradas válidas
    else:
      print("\n Opção inválida. Por favor, tente novamente.")

main()